In [11]:
import pickle
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [65]:
train_path = 'your path to train file'
with open(train_path, 'rb') as f:
    train = pickle.load(f)

test_path = 'your path to test file'
with open(test_path, 'rb') as f:
    test = pickle.load(f)

In [22]:
def add_definitions(elem):
    if elem['case'] == 'predict_hypernym':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
      #  elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'predict_multiple_hypernyms':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()

    elif elem['case'] == 'simple_triplet_grandparent':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
    elif elem['case'] == 'only_child_leaf':
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
        elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'simple_triplet_2parent':
        elem['1parent_def'] = wn.synset(elem['parents'][0]).definition()
        elem['2parent_def'] = wn.synset(elem['parents'][1]).definition()
    else:
        elem['parent_def'] = wn.synset(elem['parents']).definition()

def cut_lemmas(lemmas):
    max_lemmas = min(len(lemmas), 8)
    return lemmas[:max_lemmas]

max_len = 0
def add_lemmas(elem):
    global max_len
    if elem['case'] == 'predict_hypernym':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['children'].replace(' ', '_')).lemmas()])
        elem['child_def'] = ', '.join(lemmas)
    elif elem['case'] == 'simple_triplet_grandparent':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['children'].replace(' ', '_')).lemmas()])
        elem['child_def'] = ', '.join(lemmas)
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['grandparents']).lemmas()])
        elem['grandparent_def'] = ', '.join(lemmas)
    elif elem['case'] == 'only_child_leaf':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['grandparents']).lemmas()])
        elem['grandparent_def'] = ', '.join(lemmas)

        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents']).lemmas()])
        elem['parent_def'] = ', '.join(lemmas)
    elif elem['case'] == 'simple_triplet_2parent':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents'][0]).lemmas()])
        elem['1parent_def'] = ', '.join(lemmas)
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents'][1]).lemmas()])
        elem['2parent_def'] = ', '.join(lemmas)
    else:
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents']).lemmas()])
        elem['parent_def'] = ', '.join(lemmas)
    if len(lemmas) > max_len:
        max_len = len(lemmas)

In [27]:
for i, elem in enumerate(train):
    try:
        add_definitions(elem)
    except:
        print(i, elem)
        train.remove(elem)

In [29]:
counter = 0
for i, elem in enumerate(test):
    try:
        add_definitions(elem)
    except:
        print(i, elem)
        counter += 1
        test.remove(elem)

print(counter)

0


In [32]:
train_out = 'path to save train with lemmas'
test_out = 'path to save test with lemmas'

with open(train_out, 'wb') as f:
    pickle.dump(train, f)

with open(test_out, 'wb') as f:
    pickle.dump(test, f)